In [1]:
import requests
import time
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

### Alle Daten von der Datenbank Value Tabelle laden

In [ ]:
# Verbindung zur Datenbank herstellen
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage
select_query = "SELECT * FROM value;"

# Abfrage ausführen
cursor.execute(select_query)

# Ergebnis abrufen
rows = cursor.fetchall()

# Ergebnis in ein Pandas DataFrame umwandeln
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Ergebnis anzeigen
print(df)

# Cursor und Verbindung schließen
cursor.close()
connection.close()

In [4]:
# Anzahl Einträge je Sensebox ermitteln
senseid_counts = df['senseid_fk'].value_counts()
senseid_counts

5ae4a726223bd80019a367a6    262099
5b4d11485dc1ec001b5452c7    261212
605f498077a88b001bba3dc0    253284
590e0b0a51d3460011c725c4    241607
5c08379b1c28f9001a3f580c    237442
5dde9523ba7944001da4150e    216101
64722d1c9be0580007f776d9    125060
65e8d93acbf5700007f920ca     23074
Name: senseid_fk, dtype: int64


### Daten für alle Senseboxen definieren

In [5]:
#Messstationen definieren
boxes_dic = {
    "Mersch Luxembourg" : "5ae4a726223bd80019a367a6",
    "Ettlingen Deutschland" : "5dde9523ba7944001da4150e",
    "Stuttgart Deutschland" : "5c08379b1c28f9001a3f580c",
    "München Deutschland" : "5b4d11485dc1ec001b5452c7",
    "Berlin Deutschland" : "65e8d93acbf5700007f920ca",
    "Hamburg Deutschland" : "590e0b0a51d3460011c725c4",
    "Dortmund Deutschland" : "605f498077a88b001bba3dc0",
    "Jena Deutschland" : "64722d1c9be0580007f776d9"
}

In [6]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_messwerte = pd.DataFrame(columns=["time", "temperature","luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])
df_id = pd.DataFrame(columns=["temperature_id","luftfeuchtigkeit_id", "pm10_id", "pm2_5_id", "senseid_fk"])

### Alle SensorID holen von den Senseboxen

In [7]:

new_data = []  # A list to collect new data
for box_id, box_value in boxes_dic.items():
    
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 

        temperature_value = None
        pm10_value = None
        pm25_value = None
        rel_Luftfeuchte_value = None

        # Traverse through the 'sensors' list
        for sensor in box.get('sensors', []):
            # Check if the sensor is for Temperature
            if sensor.get('title') == 'Temperatur':
                # Access the temperature value
                temperature_value = sensor.get('_id', {})
                

            if sensor.get('title') == 'PM10':
                # Access the PM10 value
                pm10_value = sensor.get('_id', {})

            if sensor.get('title') == 'PM2.5':
                # Access the PM2.5 value
                pm25_value = sensor.get('_id', {})

            if sensor.get('title') == 'rel. Luftfeuchte':
                # Access the relative humidity value
                rel_Luftfeuchte_value = sensor.get('_id', {})
                
                break  # Exit the loop once the temperature is found
            # Append new data to the list
        new_data.append({"temperature_id": temperature_value,
                        "luftfeuchtigkeit_id": rel_Luftfeuchte_value, "pm10_id": pm10_value, "pm2_5_id": pm25_value,
                        "senseid_fk": box_value})
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)

new_df = pd.DataFrame(new_data)
    
    # Append the new DataFrame to the existing DataFrame
df_id = pd.concat([df_id, new_df], ignore_index=True)

In [8]:
df_id

,temperature_id,luftfeuchtigkeit_id,pm10_id,pm2_5_id,senseid_fk
0,5ae4a726223bd80019a367a8,5ae4a726223bd80019a367a7,5ae4a726223bd80019a367aa,5ae4a726223bd80019a367a9,5ae4a726223bd80019a367a6
1,5dde9523ba7944001da41511,5dde9523ba7944001da41510,5dde9523ba7944001da41513,5dde9523ba7944001da41512,5dde9523ba7944001da4150e
2,5c08379b1c28f9001a3f580e,5c08379b1c28f9001a3f580d,5c08379b1c28f9001a3f5810,5c08379b1c28f9001a3f580f,5c08379b1c28f9001a3f580c
3,5b4d11485dc1ec001b5452ca,5b4d11485dc1ec001b5452c9,5b4d11485dc1ec001b5452cc,5b4d11485dc1ec001b5452cb,5b4d11485dc1ec001b5452c7
4,65e8d93acbf5700007f920cb,65e8d93acbf5700007f920cc,None,None,65e8d93acbf5700007f920ca
5,592469ca51d34600119bf21f,592469ca51d34600119bf220,590e0b0a51d3460011c725c5,590e0b0a51d3460011c725c6,590e0b0a51d3460011c725c4
6,605f49bd77a88b001bba595a,None,None,None,605f498077a88b001bba3dc0
7,64722d1c9be0580007f776dc,64722d1c9be0580007f776db,64722d1c9be0580007f776de,64722d1c9be0580007f776dd,64722d1c9be0580007f776d9


### Historie der letzten 10000 Daten von der API holen

In [9]:
dates = {
    '2022-01-01T00:00:00.000Z': '2022-01-31T23:59:59.000Z',
    '2022-02-01T00:00:00.000Z': '2022-02-28T23:59:59.000Z',
    '2022-03-01T00:00:00.000Z': '2022-03-31T23:59:59.000Z',
    '2022-04-01T00:00:00.000Z': '2022-04-30T23:59:59.000Z',
    '2022-05-01T00:00:00.000Z': '2022-05-31T23:59:59.000Z',
    '2022-06-01T00:00:00.000Z': '2022-06-30T23:59:59.000Z',
    '2022-07-01T00:00:00.000Z': '2022-07-31T23:59:59.000Z',
    '2022-08-01T00:00:00.000Z': '2022-08-31T23:59:59.000Z',
    '2022-09-01T00:00:00.000Z': '2022-09-30T23:59:59.000Z',
    '2022-10-01T00:00:00.000Z': '2022-10-31T23:59:59.000Z',
    '2022-11-01T00:00:00.000Z': '2022-11-30T23:59:59.000Z',
    '2022-12-01T00:00:00.000Z': '2022-12-31T23:59:59.000Z',
    '2023-01-01T00:00:00.000Z': '2023-01-31T23:59:59.000Z',
    '2023-02-01T00:00:00.000Z': '2023-02-28T23:59:59.000Z',
    '2023-03-01T00:00:00.000Z': '2023-03-31T23:59:59.000Z',
    '2023-04-01T00:00:00.000Z': '2023-04-30T23:59:59.000Z',
    '2023-05-01T00:00:00.000Z': '2023-05-31T23:59:59.000Z',
    '2023-06-01T00:00:00.000Z': '2023-06-30T23:59:59.000Z',
    '2023-07-01T00:00:00.000Z': '2023-07-31T23:59:59.000Z',
    '2023-08-01T00:00:00.000Z': '2023-08-31T23:59:59.000Z',
    '2023-09-01T00:00:00.000Z': '2023-09-30T23:59:59.000Z',
    '2023-10-01T00:00:00.000Z': '2023-10-31T23:59:59.000Z',
    '2023-11-01T00:00:00.000Z': '2023-11-30T23:59:59.000Z',
    '2023-12-01T00:00:00.000Z': '2023-12-31T23:59:59.000Z',
    '2024-01-01T00:00:00.000Z': '2024-01-31T23:59:59.000Z',
    '2024-02-01T00:00:00.000Z': '2024-02-29T23:59:59.000Z',  
    '2024-03-01T00:00:00.000Z': '2024-03-31T23:59:59.000Z',
    '2024-04-01T00:00:00.000Z': '2024-04-30T23:59:59.000Z'

}


In [10]:
df_messwerte = pd.DataFrame(columns=["time", "temperature", "luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])

# Erstelle eine Liste, um die Dictionaries zu speichern
list_of_dicts = []

# Iteriere über jede Zeile im DataFrame, um die Liste der Dictionaries zu erstellen
for _, row in df_id.iterrows():
    senseid_fk = row['senseid_fk']
    sensors = {
        'temperature': row['temperature_id'],
        'luftfeuchtigkeit': row['luftfeuchtigkeit_id'],
        'pm10': row['pm10_id'],
        'pm2_5': row['pm2_5_id']
    }
    sensor_dict = {senseid_fk: sensors}
    list_of_dicts.append(sensor_dict)

for start_date, end_date in dates.items():

    # Iteriere über jede Box im Dictionary
    for box in list_of_dicts:
        for senseid_fk, sensors in box.items():
            df_box = pd.DataFrame()  # Initialisiere den DataFrame für die aktuelle Box
            
            for sensor_type, sensor_id in sensors.items():
                if sensor_id is not None:  # Überprüfen, ob der Sensor vorhanden ist
                    try:

                        response = requests.get(f'https://api.opensensemap.org/boxes/{senseid_fk}/data/{sensor_id}?from-date={start_date}&to-date={end_date}&download=true&format=json')
                        data = response.json()
                        
                        if data:  # Überprüfen, ob Daten vorhanden sind
                            temp_df = pd.DataFrame(data)
                            temp_df['senseid_fk'] = senseid_fk
                            temp_df = temp_df.rename(columns={'createdAt': 'time', 'value': sensor_type})
                            
                            if df_box.empty:
                                df_box = temp_df[['time', sensor_type, 'senseid_fk']]
                            else:
                                df_box = pd.merge(df_box, temp_df[['time', sensor_type]], on='time', how='outer')
                    except:
                        pass
            # Zusammenführen der aktuellen Box-Daten mit den bisherigen Daten
            df_messwerte = pd.concat([df_messwerte, df_box], ignore_index=True)

    df_messwerte_cleaned = df_messwerte.dropna(subset=['senseid_fk'])

In [11]:
df_messwerte_cleaned

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk


In [24]:
df_messwerte_cleaned['time'] = df_messwerte_cleaned['time'].str[:-1]  # Entfernt das 'Z' am Ende
df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'], format='%Y-%m-%dT%H:%M:%S.%f')


C:\Users\conta\AppData\Local\Temp\ipykernel_17644\3397160356.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messwerte_cleaned['time'] = df_messwerte_cleaned['time'].str[:-1]  # Entfernt das 'Z' am Ende
C:\Users\conta\AppData\Local\Temp\ipykernel_17644\3397160356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'], format='%Y-%m-%dT%H:%M:%S.%f')


### Historie Daten in die Datenbank hochladen

In [29]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_messwerte_cleaned.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")

Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich in die Datenbank eingefügt.


### Input in die Datenbank + Filterung nur Daten vor 15.5. 16.32Uhr

Hole Daten vom 01.05.2024 bis zum 16.05.2024

In [ ]:
df_messwerte = pd.DataFrame(columns=["time", "temperature", "luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])

# Erstelle eine Liste, um die Dictionaries zu speichern
list_of_dicts = []

# Iteriere über jede Zeile im DataFrame, um die Liste der Dictionaries zu erstellen
for _, row in df_id.iterrows():
    senseid_fk = row['senseid_fk']
    sensors = {
        'temperature': row['temperature_id'],
        'luftfeuchtigkeit': row['luftfeuchtigkeit_id'],
        'pm10': row['pm10_id'],
        'pm2_5': row['pm2_5_id']
    }
    sensor_dict = {senseid_fk: sensors}
    list_of_dicts.append(sensor_dict)

# Iteriere über jede Box im Dictionary
for box in list_of_dicts:
    for senseid_fk, sensors in box.items():
        df_box = pd.DataFrame()  # Initialisiere den DataFrame für die aktuelle Box
        
        for sensor_type, sensor_id in sensors.items():
            if sensor_id is not None:  # Überprüfen, ob der Sensor vorhanden ist
                response = requests.get(f'https://api.opensensemap.org/boxes/{senseid_fk}/data/{sensor_id}?from-date=2024-05-01T00:00:00.000Z&to-date=2024-05-16T20:44:01.434Z&download=true&format=json')
                data = response.json()
                
                if data:  # Überprüfen, ob Daten vorhanden sind
                    temp_df = pd.DataFrame(data)
                    temp_df['senseid_fk'] = senseid_fk
                    temp_df = temp_df.rename(columns={'createdAt': 'time', 'value': sensor_type})
                    
                    if df_box.empty:
                        df_box = temp_df[['time', sensor_type, 'senseid_fk']]
                    else:
                        df_box = pd.merge(df_box, temp_df[['time', sensor_type]], on='time', how='outer')
        
        # Zusammenführen der aktuellen Box-Daten mit den bisherigen Daten
        df_messwerte = pd.concat([df_messwerte, df_box], ignore_index=True)

df_messwerte_cleaned = df_messwerte.dropna(subset=['senseid_fk'])

In [ ]:
df_messwerte_cleaned['time'] = df_messwerte_cleaned['time'].str[:-1]  # Entfernt das 'Z' am Ende
df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'], format='%Y-%m-%dT%H:%M:%S.%f')

In [ ]:
df_messwerte_cleaned['time'] = pd.to_datetime(df_messwerte_cleaned['time'])

In [62]:
df_messwerte_cleaned.dtypes

time                datetime64[ns]
temperature                 object
luftfeuchtigkeit            object
pm10                        object
pm2_5                       object
senseid_fk                  object
dtype: object

In [63]:
df_filtered = df_messwerte_cleaned[(df_messwerte_cleaned['time'].dt.day<=15) & 
                                   (df_messwerte_cleaned['time'].dt.hour<=16) & 
                                   (df_messwerte_cleaned['time'].dt.minute<32)]

In [64]:
df_filtered

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
180,2024-05-08 00:02:15,13.50,99.90,11.53,10.33,5ae4a726223bd80019a367a6
181,2024-05-08 00:04:43,13.50,99.90,14.27,11.87,5ae4a726223bd80019a367a6
182,2024-05-08 00:07:10,13.40,99.90,21.90,13.70,5ae4a726223bd80019a367a6
183,2024-05-08 00:09:38,13.40,99.90,21.12,16.40,5ae4a726223bd80019a367a6
184,2024-05-08 00:12:06,13.60,99.90,18.73,14.57,5ae4a726223bd80019a367a6
...,...,...,...,...,...,...
69148,2024-05-15 16:24:14,24.66,28.14,3.15,1.33,64722d1c9be0580007f776d9
69149,2024-05-15 16:25:55,24.56,28.30,2.78,1.50,64722d1c9be0580007f776d9
69150,2024-05-15 16:27:35,24.36,28.50,5.05,1.45,64722d1c9be0580007f776d9
69151,2024-05-15 16:29:17,24.48,28.23,4.15,1.60,64722d1c9be0580007f776d9


In [ ]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_filtered.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")

### Neusten Werte von jeder Sensebox von der API holen

In [ ]:
new_data = []  # A list to collect new data
for box_id, box_value in boxes_dic.items():
    
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 

        temperature_value = None
        pm10_value = None
        pm25_value = None
        rel_Luftfeuchte_value = None

        # Traverse through the 'sensors' list
        for sensor in box.get('sensors', []):
            # Check if the sensor is for Temperature
            if sensor.get('title') == 'Temperatur':
                # Access the temperature value
                temperature_value = sensor.get('lastMeasurement', {}).get('value')
            
            if sensor.get('title') == 'PM10':
                # Access the PM10 value
                pm10_value = sensor.get('lastMeasurement', {}).get('value')

            if sensor.get('title') == 'PM2.5':
                # Access the PM2.5 value
                pm25_value = sensor.get('lastMeasurement', {}).get('value')

            if sensor.get('title') == 'rel. Luftfeuchte':
                # Access the relative humidity value
                rel_Luftfeuchte_value = sensor.get('lastMeasurement', {}).get('value')
                
                break  # Exit the loop once the temperature is found
            
        # Append new data to the list
        new_data.append({"temperature": temperature_value, "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                        "luftfeuchtigkeit": rel_Luftfeuchte_value, "pm10": pm10_value, "pm2_5": pm25_value,
                        "senseid_fk": box_value})
    
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    
    # Convert the new data list to a DataFrame
new_df = pd.DataFrame(new_data)
    
    # Append the new DataFrame to the existing DataFrame
df_messwerte = pd.concat([df_messwerte, new_df], ignore_index=True)
df_messwerte = df_messwerte.fillna(0)

# Convert specific columns to float64, handling non-numeric values
columns_to_convert = ["temperature", "luftfeuchtigkeit", "pm10", "pm2_5"]
for column in columns_to_convert:
    df_messwerte[column] = pd.to_numeric(df_messwerte[column], errors='coerce')

df_messwerte = df_messwerte.round(2)


In [9]:
df_messwerte

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-24 23:37:43,0,0,0,0,5ae4a726223bd80019a367a6
1,2024-05-24 23:37:43,0,0,0,0,5dde9523ba7944001da4150e
2,2024-05-24 23:37:43,0,0,0,0,5c08379b1c28f9001a3f580c
3,2024-05-24 23:37:43,0,0,0,0,5b4d11485dc1ec001b5452c7
4,2024-05-24 23:37:43,0,0,0,0,65e8d93acbf5700007f920ca
5,2024-05-24 23:37:43,0,0,0,0,590e0b0a51d3460011c725c4
6,2024-05-24 23:37:44,0,0,0,0,605f498077a88b001bba3dc0
7,2024-05-24 23:37:44,0,0,0,0,64722d1c9be0580007f776d9
8,2024-05-24 23:38:29,None,None,None,None,5ae4a726223bd80019a367a6
9,2024-05-24 23:38:30,None,None,None,None,5dde9523ba7944001da4150e


In [7]:
df_messwerte["time"] = pd.to_datetime(df_messwerte["time"])

In [8]:
df_messwerte.dtypes

time                datetime64[ns]
temperature                float64
luftfeuchtigkeit           float64
pm10                       float64
pm2_5                      float64
senseid_fk                  object
dtype: object

### Neusten Werte in die Datenbank hochladen

In [9]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_messwerte.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich in die Datenbank eingefügt.


### Zweites DataFrame für Daten der Messstationen erstellen

In [10]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_stations = pd.DataFrame(columns=["Ort","Land","Lat", "Lon", "Hoehe", "senseID"])

### Alle Metadaten der Senseboxen holen

In [11]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    temperature_value = None
    lat = None
    lon = None
    hoehe = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None
    ort_Land = box_id.split()

    # Extrahiere Koordinaten
    current_location = box.get('currentLocation', {})
    if current_location:
        coordinates = current_location.get('coordinates')
        if coordinates:
            lon = coordinates[0]
            lat = coordinates[1]
            height = coordinates[2] if len(coordinates) >= 3 else None

        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Ort": ort_Land[0], "Land":ort_Land[1], "Lat": lat,"Lon":lon,"Hoehe":height, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df_stations = pd.concat([df_stations, new_df], ignore_index=True)

FUCK YOU SENSEBOX NO WORKING TODAY590e0b0a51d3460011c725c4


In [12]:
df_stations

,Ort,Land,Lat,Lon,Hoehe,senseID
0,Mersch,Luxembourg,49.754663,6.105682,250.0,5ae4a726223bd80019a367a6
1,Ettlingen,Deutschland,48.926209,8.374409,4.0,5dde9523ba7944001da4150e
2,Stuttgart,Deutschland,48.779610,9.213723,280.0,5c08379b1c28f9001a3f580c
3,München,Deutschland,48.146546,11.576033,20.0,5b4d11485dc1ec001b5452c7
4,Berlin,Deutschland,52.511270,13.397356,45.0,65e8d93acbf5700007f920ca
5,Hamburg,Deutschland,52.511270,13.397356,45.0,590e0b0a51d3460011c725c4
6,Dortmund,Deutschland,51.497350,7.494606,NaN,605f498077a88b001bba3dc0
7,Jena,Deutschland,50.934561,11.589701,146.0,64722d1c9be0580007f776d9
